In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from joblib import dump
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from joblib import dump
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.metrics import mean_absolute_error as mae


In [3]:
X_train = pd.read_csv('../data/processed/X_train.csv')
X_val   = pd.read_csv('../data/processed/X_val.csv'  )
X_test  = pd.read_csv('../data/processed/X_test.csv' )
y_train = pd.read_csv('../data/processed/y_train.csv')
y_val   = pd.read_csv('../data/processed/y_val.csv'  )
y_test  = pd.read_csv('../data/processed/y_test.csv' )

In [4]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

In [5]:
poly = PolynomialFeatures(degree=2)

In [6]:
X_train_poly = poly.fit_transform(X_train_scaled)
X_val_poly = poly.transform(X_val_scaled)

In [7]:
log_reg = LogisticRegression(solver="liblinear",max_iter=1000)

param_grid = {
    "C": [100, 10, 1.0, 0.1, 0.01],
    "penalty":["l1","l2"]
}

In [8]:
y_train = y_train.values.ravel()

In [ ]:
cv_log = StratifiedKFold(n_splits=5)

grid = GridSearchCV(log_reg, param_grid, cv=cv_log, refit=True, scoring='roc_auc', verbose=2)
grid.fit(X_train_poly, y_train)

print("Best hyperparameters:", grid.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [ ]:
best_log_reg = grid.best_estimator_
y_pred_proba = best_log_reg.predict_proba(X_val_poly)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print("Validation ROC AUC:", roc_auc)

In [ ]:
model1 = LogisticRegression(solver="liblinear",C=0.1, penalty= "l1")

model1.fit(X_train_scaled, y_train)